Load large data for network from file

In [1]:
import tensorflow as tf
import pickle
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


In [3]:

n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 2
batch_size = 100
hm_epochs = 10


In [4]:
n_classes = 2

batch_size = 32
total_batches = int(1600000/batch_size)
hm_epochs = 10


In [5]:
class TfAnn(object):
    
    def __init__(self):
        self.hidden_1_layer = {'weights':[],'biases':[]}
        self.hidden_2_layer = {'weights':[],'biases':[]}
        self.hidden_3_layer = {'weights':[],'biases':[]}
        self.output_layer = {'weights':[],'biases':[]}
        self.np_hidden_1_layer={"weights":[],"biases":[]}
        self.np_hidden_2_layer={"weights":[],"biases":[]}
        self.np_hidden_3_layer={"weights":[],"biases":[]}
        self.np_output_layer={"weights":[],"biases":[]}
    
    def init_empty(self,size,n_nodes_hl1,n_nodes_hl2,n_nodes_hl3,n_classes):
        self.hidden_1_layer = {'weights':tf.Variable(tf.random_normal([size, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

        self.hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

        self.hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

        self.output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                        'biases':tf.Variable(tf.random_normal([n_classes]))}


    def init_values(self,size,l1_weights,l2_weights,l3_weights,out_weights,l1_biases,l2_biases,l3_biases,out_biases):
        self.hidden_1_layer = {'weights':tf.Variable(l1_weights),'biases':tf.Variable(l1_biases)}
        self.hidden_2_layer = {'weights':tf.Variable(l2_weights),'biases':tf.Variable(l2_biases)}
        self.hidden_3_layer = {'weights':tf.Variable(l3_weights),'biases':tf.Variable(l3_biases)}
        self.output_layer = {'weights':tf.Variable(out_weights),'biases':tf.Variable(out_biases)}


    def create(self,data):
        # This is the heart of the ann where multiply the data by the wights to the layers 
        l1 = tf.add(tf.matmul(data,self.hidden_1_layer['weights']), self.hidden_1_layer['biases'])
        l1 = tf.nn.relu(l1)

        l2 = tf.add(tf.matmul(l1,self.hidden_2_layer['weights']), self.hidden_2_layer['biases'])
        l2 = tf.nn.relu(l2)

        l3 = tf.add(tf.matmul(l2,self.hidden_3_layer['weights']), self.hidden_3_layer['biases'])
        l3 = tf.nn.relu(l3)

        output =  tf.add(tf.matmul(l3,self.output_layer['weights']) , self.output_layer['biases'])

        return output
    
    def extract(self):
        self.np_hidden_1_layer["weights"] = neural_network_model.hidden_1_layer["weights"].eval()
        self.np_hidden_2_layer["weights"] = neural_network_model.hidden_2_layer["weights"].eval()
        self.np_hidden_3_layer["weights"] = neural_network_model.hidden_3_layer["weights"].eval()
        self.np_output_layer["weights"] = neural_network_model.output_layer["weights"].eval()
        self.np_hidden_1_layer["biases"] = neural_network_model.hidden_1_layer["biases"].eval()
        self.np_hidden_2_layer["biases"] = neural_network_model.hidden_2_layer["biases"].eval()
        self.np_hidden_3_layer["biases"] = neural_network_model.hidden_3_layer["biases"].eval()
        self.np_output_layer["biases"] = neural_network_model.output_layer["biases"].eval()
        

In [18]:
def train_neural_network_interactive(neural_network_model):
    x= tf.placeholder('float')
    y = tf.placeholder('float')
    prediction = neural_network_model.create(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    hm_epochs = 10
    sess = tf.InteractiveSession()
    init = tf.initialize_all_variables()
    sess.run(init)
    epoch = 1
    while epoch <= hm_epochs:
        epoch_loss = 0
        with open('Data/lexicon-2500-2638.pickle','rb') as f:
            lexicon = pickle.load(f)
        with open('Data/train_set_shuffled008.csv', buffering=20000, encoding='latin-1') as f:
            batch_x = []
            batch_y = []
            batches_run = 0
            for line in f:
                label = line.split(':::')[0]
                tweet = line.split(':::')[1]
                current_words = word_tokenize(tweet.lower())
                current_words = [lemmatizer.lemmatize(i) for i in current_words]

                features = np.zeros(len(lexicon))

                for word in current_words:
                    if word.lower() in lexicon:
                        index_value = lexicon.index(word.lower())
                        # OR DO +=1, test both
                        features[index_value] += 1
                line_x = list(features)
                line_y = eval(label)
                batch_x.append(line_x)
                batch_y.append(line_y)
                if len(batch_x) >= batch_size:
                    _, c = sess.run([optimizer, cost], feed_dict={x: np.array(batch_x),
                                                              y: np.array(batch_y)})
                    epoch_loss += c
                    batch_x = []
                    batch_y = []
                    batches_run +=1
                    if batches_run%10000==0:
                        print('Batch run:',batches_run,'/',total_batches,'| Epoch:',epoch,'| Batch Loss:',c,)
                    if batches_run%1000==0:
                        print(batches_run,end=":")
                        print(epoch,end=" ")

    neural_network_model.extract()
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    print('Accuracy:',accuracy.eval({x:test_x, y:test_y}))

In [ ]:
neural_network_model = TfAnn()
neural_network_model.init_empty(2638,n_nodes_hl1,n_nodes_hl2,n_nodes_hl3,n_classes)
train_neural_network_interactive(neural_network_model)

1000:1 2000:1 3000:1 4000:1 5000:1 6000:1 7000:1 8000:1 9000:1 Batch run: 10000 / 50000 | Epoch: 1 | Batch Loss: 156.84756
10000:1 11000:1 12000:1 13000:1 14000:1 15000:1 16000:1 17000:1 18000:1 19000:1 Batch run: 20000 / 50000 | Epoch: 1 | Batch Loss: 17.1883
20000:1 21000:1 22000:1 23000:1 24000:1 25000:1 26000:1 27000:1 28000:1 29000:1 Batch run: 30000 / 50000 | Epoch: 1 | Batch Loss: 10.417189
30000:1 31000:1 32000:1 33000:1 34000:1 35000:1 36000:1 37000:1 38000:1 39000:1 Batch run: 40000 / 50000 | Epoch: 1 | Batch Loss: 2.8684473
40000:1 41000:1 42000:1 43000:1 44000:1 45000:1 46000:1 47000:1 48000:1 49000:1 Batch run: 50000 / 50000 | Epoch: 1 | Batch Loss: 2.5914087
50000:1 51000:1 52000:1 53000:1 54000:1 55000:1 56000:1 57000:1 58000:1 59000:1 Batch run: 60000 / 50000 | Epoch: 1 | Batch Loss: 2.7381139
60000:1 61000:1 62000:1 63000:1 64000:1 65000:1 66000:1 67000:1 68000:1 69000:1 Batch run: 70000 / 50000 | Epoch: 1 | Batch Loss: 3.6663966
70000:1 71000:1 72000:1 73000:1 74000:1